# Text-to-Image AI Service in Google Colab

This notebook sets up and runs a text-to-image generation service using Stable Diffusion with TensorFlow and KerasCV.

## Requirements
- Enable GPU runtime: Runtime → Change runtime type → Hardware accelerator → GPU
- High-RAM runtime recommended for better performance

## Features
- 🎨 Stable Diffusion image generation
- 🚀 FastAPI REST API
- 🌐 Public URL via ngrok
- 📱 Interactive web interface

## Step 1: Check GPU and Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

# Check Python version
import sys
print(f"Python version: {sys.version}")

# Set up environment variables
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['COLAB_GPU'] = '1'

print("✅ Environment check complete")

## Step 2: Install Dependencies

In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq libgl1-mesa-glx libglib2.0-0 libsm6 libxext6 libxrender-dev libgomp1

print("✅ System dependencies installed")

In [ ]:
# Install Python dependencies
!pip install -q tensorflow>=2.15.0
!pip install -q keras-cv>=0.6.0
!pip install -q fastapi>=0.104.0
!pip install -q uvicorn[standard]>=0.24.0
!pip install -q python-multipart>=0.0.6
!pip install -q pillow>=10.0.0
!pip install -q pydantic>=2.5.0
!pip install -q pydantic-settings>=2.1.0
!pip install -q python-dotenv>=1.0.0
!pip install -q structlog>=23.2.0
!pip install -q nest-asyncio
!pip install -q pyngrok
!pip install -q scipy  # For image enhancement

print("✅ Python dependencies installed")

## Step 3: Setup ngrok Authentication (REQUIRED)

**IMPORTANT**: ngrok now requires authentication even for free usage.

1. **Sign up for free**: Go to https://dashboard.ngrok.com/signup
2. **Get your auth token**: Visit https://dashboard.ngrok.com/get-started/your-authtoken  
3. **Copy the token**: It looks like `2abc123_def456ghi789jkl...`
4. **Replace the token below**: Change `YOUR_NGROK_AUTH_TOKEN_HERE` to your actual token

In [ ]:
# Set up ngrok authentication (REQUIRED)
# Get your auth token from: https://dashboard.ngrok.com/get-started/your-authtoken

from pyngrok import ngrok, conf

# REPLACE THIS with your actual ngrok auth token:
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN_HERE"

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTH_TOKEN_HERE":
    print("❌ ERROR: Please set your ngrok auth token!")
    print("1. Go to: https://dashboard.ngrok.com/signup (free signup)")
    print("2. Get token: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("3. Replace YOUR_NGROK_AUTH_TOKEN_HERE above with your token")
    raise ValueError("ngrok auth token required")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok authentication configured successfully!")

## Step 4: Create and Run the Text-to-Image Service

In [ ]:
import os
import time
import asyncio
import nest_asyncio
from typing import Optional, List
import tensorflow as tf
import keras_cv
import numpy as np
from PIL import Image
import io
import base64
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import uvicorn
from pyngrok import ngrok
import threading

# Enable nested asyncio for Colab
nest_asyncio.apply()

print("✅ Imports complete")

In [ ]:
# Configure TensorFlow for Colab
def setup_tensorflow():
    """Configure TensorFlow for Colab environment."""
    gpus = tf.config.experimental.list_physical_devices("GPU")
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.keras.mixed_precision.set_global_policy("mixed_float16")
            print(f"✅ Configured {len(gpus)} GPU(s) with mixed precision")
        except RuntimeError as e:
            print(f"⚠️ GPU setup error: {e}")
    else:
        print("⚠️ No GPU detected, using CPU (will be slow)")
    return len(gpus) > 0

gpu_available = setup_tensorflow()

In [ ]:
# Define enhanced API models with better defaults
class ImageRequest(BaseModel):
    prompt: str = Field(..., min_length=1, max_length=500, description="Text description of the image")
    num_steps: int = Field(35, ge=15, le=50, description="Number of diffusion steps (higher = better quality)")
    guidance_scale: float = Field(12.0, ge=1.0, le=20.0, description="How closely to follow the prompt (higher = more adherence)")
    seed: Optional[int] = Field(None, ge=0, description="Random seed for reproducible results")

class ImageResponse(BaseModel):
    image_base64: str
    prompt: str
    enhanced_prompt: str
    generation_time: float
    parameters: dict
    quality_metrics: dict

print("✅ Enhanced API models defined")

In [ ]:
# Create the Enhanced Stable Diffusion model class
class ColabStableDiffusion:
    """Enhanced Stable Diffusion model for better quality generation."""

    def __init__(self):
        self.model = None
        self.load_model()

    def load_model(self):
        """Load Stable Diffusion model with optimized settings."""
        print("🔄 Loading Enhanced Stable Diffusion model (this may take a few minutes)...")

        try:
            # Use higher resolution and optimized settings
            self.model = keras_cv.models.StableDiffusion(
                img_width=512,
                img_height=512,
                jit_compile=False,  # Keep disabled for Colab compatibility
            )
            
            # Warm up the model with a dummy generation for better performance
            print("🔄 Warming up model for optimal performance...")
            try:
                _ = self.model.text_to_image(
                    prompt="test",
                    batch_size=1,
                    num_steps=5,  # Minimal steps for warmup
                )
                print("✅ Model warmed up successfully!")
            except Exception as warmup_error:
                print(f"⚠️ Warmup failed (non-critical): {warmup_error}")
            
            print("✅ Enhanced Stable Diffusion model loaded successfully!")
        except Exception as e:
            print(f"❌ Model loading failed: {e}")
            raise

    def generate_image(self, prompt: str, num_steps: int = 35,
                      guidance_scale: float = 12.0, seed: Optional[int] = None):
        """Generate high-quality image from text prompt with enhanced settings."""
        if self.model is None:
            raise RuntimeError("Model not loaded")

        if seed is not None:
            tf.random.set_seed(seed)
            np.random.seed(seed)

        try:
            # Enhanced prompt preprocessing
            enhanced_prompt = self._enhance_prompt(prompt)
            print(f"🎨 Generating high-quality image for: {enhanced_prompt[:60]}...")
            print(f"⚙️ Settings: {num_steps} steps, guidance {guidance_scale}")
            
            start_time = time.time()

            # Enhanced generation parameters for better quality
            generation_params = {
                "prompt": enhanced_prompt,
                "batch_size": 1,
                "num_steps": max(num_steps, 25),  # Minimum 25 steps for quality
                "seed": seed,
            }

            # Try with guidance_scale parameter (newer versions)
            try:
                generation_params["guidance_scale"] = guidance_scale
                generated_images = self.model.text_to_image(**generation_params)
            except TypeError as e:
                if "guidance_scale" in str(e):
                    print("⚠️ Using unconditional_guidance_scale parameter for older KerasCV version")
                    generation_params.pop("guidance_scale")
                    generation_params["unconditional_guidance_scale"] = guidance_scale
                    try:
                        generated_images = self.model.text_to_image(**generation_params)
                    except TypeError:
                        print("⚠️ Using basic parameters without guidance scale")
                        generation_params.pop("unconditional_guidance_scale")
                        generated_images = self.model.text_to_image(**generation_params)
                else:
                    raise e

            # Enhanced image processing for better quality
            img_array = generated_images[0]

            # Debug output
            print(f"📊 Raw image shape: {img_array.shape}")
            print(f"📊 Raw value range: {img_array.min():.4f} to {img_array.max():.4f}")
            print(f"📊 Raw dtype: {img_array.dtype}")

            # Ensure correct shape
            if len(img_array.shape) == 4:
                img_array = img_array[0]  # Remove batch dimension
            
            # Convert to float32 for precise processing
            img_array = img_array.astype(np.float32)

            # Enhanced normalization with better quality preservation
            if img_array.min() >= 0.0 and img_array.max() <= 1.0:
                # Standard [0,1] to [0,255] with gamma correction for better contrast
                img_array = np.power(img_array, 0.9)  # Slight gamma correction
                img_array = img_array * 255.0
                print("✅ Applied enhanced [0,1] → [0,255] normalization with gamma correction")
            elif img_array.min() >= -1.0 and img_array.max() <= 1.0:
                # Enhanced [-1,1] to [0,255] conversion
                img_array = (img_array + 1.0) / 2.0  # Convert to [0,1]
                img_array = np.power(img_array, 0.9)  # Gamma correction
                img_array = img_array * 255.0
                print("✅ Applied enhanced [-1,1] → [0,255] normalization with gamma correction")
            else:
                # Robust normalization with contrast enhancement
                img_min, img_max = img_array.min(), img_array.max()
                if img_max > img_min:
                    img_array = (img_array - img_min) / (img_max - img_min)
                    # Apply contrast enhancement
                    img_array = np.power(img_array, 0.9)
                    img_array = img_array * 255.0
                else:
                    img_array = np.zeros_like(img_array)
                print(f"✅ Applied custom normalization with contrast enhancement")

            # Enhanced post-processing for better quality
            img_array = self._enhance_image_quality(img_array)
            
            # Final clipping and conversion
            img_array = np.clip(img_array, 0, 255).astype(np.uint8)
            
            # Quality check
            unique_values = len(np.unique(img_array))
            print(f"📊 Final image: {img_array.shape}, {unique_values} unique values")
            
            if unique_values < 50:
                print("⚠️ Warning: Low color diversity detected")
            else:
                print("✅ Good color diversity achieved")

            # Create high-quality PIL image
            pil_image = Image.fromarray(img_array, mode='RGB')
            
            generation_time = time.time() - start_time
            print(f"✅ High-quality image generated in {generation_time:.1f} seconds")

            return pil_image

        except Exception as e:
            print(f"❌ Generation failed: {e}")
            import traceback
            traceback.print_exc()
            raise

    def _enhance_prompt(self, prompt: str) -> str:
        """Enhance the prompt for better quality results."""
        # Add quality enhancing keywords if not already present
        quality_keywords = [
            "high quality", "detailed", "sharp", "clear", "professional",
            "8k", "hd", "masterpiece", "best quality"
        ]
        
        prompt_lower = prompt.lower()
        has_quality_terms = any(keyword in prompt_lower for keyword in quality_keywords)
        
        if not has_quality_terms:
            prompt = f"{prompt}, high quality, detailed, sharp focus, professional"
        
        # Remove common quality-degrading terms
        negative_terms = ["blurry", "low quality", "pixelated", "jpeg artifacts"]
        for term in negative_terms:
            prompt = prompt.replace(term, "").replace(term.capitalize(), "")
        
        return prompt.strip()

    def _enhance_image_quality(self, img_array: np.ndarray) -> np.ndarray:
        """Apply post-processing to enhance image quality."""
        # Slight sharpening using unsharp mask technique
        from scipy import ndimage
        
        try:
            # Create a slightly blurred version
            blurred = ndimage.gaussian_filter(img_array, sigma=0.5)
            
            # Create sharpening mask
            sharpening_strength = 0.2
            enhanced = img_array + sharpening_strength * (img_array - blurred)
            
            # Ensure values stay in valid range
            enhanced = np.clip(enhanced, 0, 255)
            
            print("✅ Applied subtle sharpening enhancement")
            return enhanced
        except ImportError:
            print("ℹ️ Scipy not available, skipping sharpening enhancement")
            return img_array
        except Exception as e:
            print(f"⚠️ Enhancement failed: {e}, using original")
            return img_array

# Initialize the enhanced model
print("🚀 Initializing Enhanced Stable Diffusion model...")
sd_model = ColabStableDiffusion()

In [ ]:
# Create FastAPI application with web interface
from fastapi.staticfiles import StaticFiles
from fastapi.responses import RedirectResponse

app = FastAPI(
    title="Enhanced Text-to-Image AI (Colab)",
    description="High-quality Stable Diffusion text-to-image generation running in Google Colab",
    version="2.0.0"
)

# Store the HTML for serving (will be updated later with enhanced interface)
import os
os.makedirs('/tmp/static', exist_ok=True)

# Mount static files
app.mount("/static", StaticFiles(directory="/tmp/static"), name="static")

@app.post("/generate", response_model=ImageResponse)
async def generate_image_endpoint(request: ImageRequest):
    """Generate high-quality image from text prompt."""
    start_time = time.time()

    try:
        # Get enhanced prompt for quality metrics
        enhanced_prompt = sd_model._enhance_prompt(request.prompt)

        # Run generation in thread pool to avoid blocking
        loop = asyncio.get_event_loop()
        pil_image = await loop.run_in_executor(
            None,
            sd_model.generate_image,
            request.prompt,
            request.num_steps,
            request.guidance_scale,
            request.seed,
        )

        # Convert to base64 with high quality PNG
        buffer = io.BytesIO()
        pil_image.save(buffer, format="PNG", optimize=False, compress_level=1)
        img_base64 = base64.b64encode(buffer.getvalue()).decode()

        generation_time = time.time() - start_time

        # Calculate quality metrics
        img_array = np.array(pil_image)
        quality_metrics = {
            "unique_colors": int(len(np.unique(img_array))),
            "image_size": pil_image.size,
            "color_channels": len(img_array.shape),
            "dynamic_range": f"{img_array.min()}-{img_array.max()}",
            "enhancement_applied": True
        }

        return ImageResponse(
            image_base64=img_base64,
            prompt=request.prompt,
            enhanced_prompt=enhanced_prompt,
            generation_time=generation_time,
            parameters={
                "num_steps": request.num_steps,
                "guidance_scale": request.guidance_scale,
                "seed": request.seed
            },
            quality_metrics=quality_metrics
        )

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Generation failed: {str(e)}")

@app.get("/health")
async def health_check():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "model_loaded": sd_model.model is not None,
        "gpu_available": gpu_available,
        "environment": "Google Colab",
        "version": "2.0.0",
        "enhancements": ["quality_mode", "gamma_correction", "sharpening", "prompt_enhancement"]
    }

@app.get("/")
async def root():
    """Root endpoint - redirect to web interface."""
    return RedirectResponse(url="/static/index.html")

print("✅ Enhanced FastAPI application created")

In [ ]:
# Enhanced web interface with better quality defaults
enhanced_web_interface_html = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Enhanced Text-to-Image AI Generator</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 20px; }
        .container { max-width: 900px; margin: 0 auto; background: rgba(255, 255, 255, 0.95); border-radius: 20px; padding: 30px; box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1); }
        .header { text-align: center; margin-bottom: 30px; }
        .header h1 { color: #333; font-size: 2.5em; margin-bottom: 10px; background: linear-gradient(45deg, #667eea, #764ba2); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text; }
        .header .subtitle { color: #666; font-size: 1.1em; }
        .quality-badge { display: inline-block; background: linear-gradient(45deg, #28a745, #20c997); color: white; padding: 5px 12px; border-radius: 15px; font-size: 0.9em; margin-top: 10px; }
        .form-group { margin-bottom: 20px; }
        .form-group label { display: block; margin-bottom: 8px; color: #333; font-weight: 600; }
        .form-group input, .form-group textarea { width: 100%; padding: 12px; border: 2px solid #ddd; border-radius: 8px; font-size: 14px; }
        .form-group textarea { min-height: 80px; resize: vertical; }
        .generate-btn { width: 100%; background: linear-gradient(45deg, #667eea, #764ba2); color: white; border: none; padding: 15px; border-radius: 10px; font-size: 16px; font-weight: 600; cursor: pointer; transition: all 0.3s; }
        .generate-btn:hover { transform: translateY(-2px); box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4); }
        .generate-btn:disabled { opacity: 0.6; cursor: not-allowed; transform: none; }
        .loading { display: none; text-align: center; margin: 20px 0; }
        .loading-spinner { border: 4px solid #f3f3f3; border-top: 4px solid #667eea; border-radius: 50%; width: 40px; height: 40px; animation: spin 1s linear infinite; margin: 0 auto 20px; }
        @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
        .result-image { max-width: 100%; border-radius: 15px; box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2); margin: 20px 0; }
        .error-message { background: #f8d7da; color: #721c24; padding: 15px; border-radius: 10px; margin: 20px 0; }
        .success-message { background: #d4edda; color: #155724; padding: 15px; border-radius: 10px; margin: 20px 0; }
        .quality-metrics { background: #e7f3ff; color: #0c5460; padding: 15px; border-radius: 10px; margin: 10px 0; font-size: 0.9em; }
        .range-group { display: flex; align-items: center; gap: 10px; }
        .range-value { background: #667eea; color: white; padding: 4px 8px; border-radius: 4px; font-size: 12px; min-width: 35px; text-align: center; }
        .quality-indicator { font-size: 0.8em; color: #666; margin-left: 10px; }
        .tips { background: #fff3cd; color: #856404; padding: 15px; border-radius: 10px; margin: 20px 0; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🎨 Enhanced AI Image Generator</h1>
            <p class="subtitle">High-quality images with AI-powered Stable Diffusion</p>
            <span class="quality-badge">✨ Enhanced Quality Mode</span>
        </div>

        <div class="tips">
            <strong>💡 Quality Tips:</strong>
            <ul style="margin: 10px 0; padding-left: 20px;">
                <li>Be specific and descriptive in your prompts</li>
                <li>Higher steps = better quality (35+ recommended)</li>
                <li>Higher guidance = closer prompt adherence</li>
                <li>Add quality terms like "detailed, sharp, professional"</li>
            </ul>
        </div>

        <form id="generateForm">
            <div class="form-group">
                <label for="prompt">🖋️ Describe your image:</label>
                <textarea id="prompt" name="prompt" placeholder="a photorealistic portrait of a majestic lion, detailed fur texture, golden hour lighting, professional photography..." required></textarea>
            </div>

            <div class="form-group">
                <label for="num_steps">🔄 Quality Steps: <span class="range-value" id="steps-value">35</span></label>
                <div class="range-group">
                    <input type="range" id="num_steps" name="num_steps" min="15" max="50" value="35" oninput="updateStepsValue(this.value)">
                    <span class="quality-indicator" id="steps-indicator">High Quality</span>
                </div>
            </div>

            <div class="form-group">
                <label for="guidance_scale">🎯 Prompt Adherence: <span class="range-value" id="guidance-value">12.0</span></label>
                <div class="range-group">
                    <input type="range" id="guidance_scale" name="guidance_scale" min="1" max="20" step="0.5" value="12.0" oninput="updateGuidanceValue(this.value)">
                    <span class="quality-indicator" id="guidance-indicator">Strong</span>
                </div>
            </div>

            <button type="submit" class="generate-btn" id="generateBtn">🚀 Generate High-Quality Image</button>
        </form>

        <div class="loading" id="loading">
            <div class="loading-spinner"></div>
            <p>🎨 Creating your high-quality masterpiece...</p>
        </div>

        <div id="results"></div>
    </div>

    <script>
        const API_BASE_URL = window.location.origin;

        function updateStepsValue(value) {
            document.getElementById('steps-value').textContent = value;
            const indicator = document.getElementById('steps-indicator');
            if (value < 20) indicator.textContent = 'Fast';
            else if (value < 30) indicator.textContent = 'Good';
            else if (value < 40) indicator.textContent = 'High Quality';
            else indicator.textContent = 'Maximum Quality';
        }

        function updateGuidanceValue(value) {
            document.getElementById('guidance-value').textContent = value;
            const indicator = document.getElementById('guidance-indicator');
            if (value < 5) indicator.textContent = 'Creative';
            else if (value < 10) indicator.textContent = 'Balanced';
            else if (value < 15) indicator.textContent = 'Strong';
            else indicator.textContent = 'Very Strong';
        }

        document.getElementById('generateForm').addEventListener('submit', async function(e) {
            e.preventDefault();

            const generateBtn = document.getElementById('generateBtn');
            const loading = document.getElementById('loading');
            const results = document.getElementById('results');

            const formData = new FormData(e.target);
            const requestData = {
                prompt: formData.get('prompt'),
                num_steps: parseInt(formData.get('num_steps')),
                guidance_scale: parseFloat(formData.get('guidance_scale'))
            };

            generateBtn.disabled = true;
            generateBtn.textContent = '🎨 Generating...';
            loading.style.display = 'block';
            results.innerHTML = '';

            try {
                const response = await fetch(`${API_BASE_URL}/generate`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(requestData)
                });

                if (!response.ok) {
                    const error = await response.json();
                    throw new Error(error.detail || `HTTP ${response.status}`);
                }

                const result = await response.json();
                
                const qualityInfo = result.quality_metrics ? `
                    <div class="quality-metrics">
                        <strong>📊 Quality Metrics:</strong><br>
                        🎨 Unique Colors: ${result.quality_metrics.unique_colors}<br>
                        📐 Image Size: ${result.quality_metrics.image_size[0]}×${result.quality_metrics.image_size[1]}<br>
                        ✨ Enhanced: ${result.quality_metrics.enhancement_applied ? 'Yes' : 'No'}<br>
                        🎯 Enhanced Prompt: "${result.enhanced_prompt}"
                    </div>
                ` : '';
                
                results.innerHTML = `
                    <div class="success-message">
                        <h3>✅ High-Quality Image Generated!</h3>
                        <p>Generation time: ${result.generation_time.toFixed(1)} seconds</p>
                        <p>Steps: ${result.parameters.num_steps} | Guidance: ${result.parameters.guidance_scale}</p>
                    </div>
                    ${qualityInfo}
                    <img src="data:image/png;base64,${result.image_base64}" alt="Generated Image" class="result-image">
                `;

            } catch (error) {
                results.innerHTML = `
                    <div class="error-message">
                        <h3>❌ Generation Failed</h3>
                        <p><strong>Error:</strong> ${error.message}</p>
                    </div>
                `;
            } finally {
                generateBtn.disabled = false;
                generateBtn.textContent = '🚀 Generate High-Quality Image';
                loading.style.display = 'none';
            }
        });
    </script>
</body>
</html>'''

# Update the static HTML file with enhanced interface
with open('/tmp/static/index.html', 'w') as f:
    f.write(enhanced_web_interface_html)

print("✅ Enhanced web interface with quality improvements created")

## Step 5: Start the Server with Public URL

In [ ]:
# Start the server with ngrok tunnel
import threading
import time
import asyncio
from uvicorn import Config, Server

def start_server():
    """Start the FastAPI server with proper async handling."""
    print("🔄 Starting FastAPI server...")
    try:
        # Create a new event loop for this thread
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
        # Configure and start the server
        config = Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = Server(config)
        
        # Run the server
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f"❌ Server startup failed: {e}")

# Start the server in a separate thread BEFORE creating ngrok tunnel
print("🚀 Starting the FastAPI server...")
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()

# Wait for server to start
print("⏳ Waiting for server to initialize...")
time.sleep(15)  # Increased wait time

# Test if server is running locally
import requests
server_ready = False
for attempt in range(5):  # Try 5 times
    try:
        test_response = requests.get("http://localhost:8000/health", timeout=5)
        if test_response.status_code == 200:
            print("✅ Server is running locally!")
            server_ready = True
            break
        else:
            print(f"⚠️ Server responded with status: {test_response.status_code}")
    except Exception as e:
        print(f"🔄 Attempt {attempt + 1}/5: Server not ready yet...")
        time.sleep(5)

if not server_ready:
    print("❌ Server failed to start properly. Please restart runtime and try again.")
else:
    # Now start ngrok tunnel (uses the auth token set earlier)
    print("🌐 Creating ngrok tunnel...")
    try:
        public_tunnel = ngrok.connect(8000)
        public_url = str(public_tunnel)  # Convert to string URL
        
        print("\n" + "="*60)
        print("🚀 TEXT-TO-IMAGE AI SERVICE IS LIVE!")
        print("="*60)
        print(f"🌐 Web Interface: {public_url}")
        print(f"🎨 Direct Access: {public_url}/static/index.html")
        print(f"📖 API Documentation: {public_url}/docs")
        print(f"❤️ Health Check: {public_url}/health")
        print(f"🔧 API Generate: {public_url}/generate")
        print("="*60)
        print("\n🎯 HOW TO USE:")
        print("1. 🖱️  Click the Web Interface link above")
        print("2. 🖋️  Enter your image description")
        print("3. ⚙️  Adjust settings as needed")
        print("4. 🚀 Click 'Generate Image'")
        print("5. ⏳ Wait 30-120 seconds for your AI masterpiece!")
        print("\n💡 TIPS:")
        print("• Be descriptive in your prompts")
        print("• Use negative prompts to avoid unwanted elements")
        print("• Lower steps = faster generation")
        print("• Higher guidance = follows prompt more closely")
        print("\n⚠️ Note: Keep this cell running to maintain the service")
        print("="*60)
        
    except Exception as e:
        print(f"❌ Failed to create ngrok tunnel: {e}")

if server_ready:
    print("\n✅ Setup complete! Click the Web Interface link above to start creating images!")
else:
    print("\n❌ Setup failed. Please restart runtime and try again.")

## Step 6: Troubleshooting & Server Check

If you're getting ngrok connection errors, run this cell to diagnose the issue:

In [ ]:
# Troubleshooting: Check if server is running properly
import requests
import time
import subprocess
import threading
import asyncio
from uvicorn import Config, Server

print("🔍 Diagnosing server status...")

# Check if port 8000 is in use
try:
    result = subprocess.run(['netstat', '-tuln'], capture_output=True, text=True)
    if ':8000' in result.stdout:
        print("✅ Port 8000 is in use (server might be running)")
    else:
        print("❌ Port 8000 is not in use (server not running)")
except:
    print("⚠️ Could not check port status")

# Test local server connection
print("\n🔍 Testing local server connection...")
try:
    response = requests.get("http://localhost:8000/health", timeout=10)
    print(f"✅ Local server is responding! Status: {response.status_code}")
    print(f"Response: {response.json()}")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to local server on port 8000")
    print("🔄 Trying to restart the server...")
    
    # Try to restart server with proper async handling
    def restart_server():
        try:
            # Create a new event loop for this thread
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
            # Configure and start the server
            config = Config(app, host="0.0.0.0", port=8000, log_level="info")
            server = Server(config)
            
            # Run the server
            loop.run_until_complete(server.serve())
        except Exception as e:
            print(f"❌ Server restart failed: {e}")
    
    server_thread = threading.Thread(target=restart_server)
    server_thread.daemon = True
    server_thread.start()
    
    print("⏳ Waiting 20 seconds for server restart...")
    time.sleep(20)
    
    # Test again
    for attempt in range(3):
        try:
            response = requests.get("http://localhost:8000/health", timeout=5)
            print(f"✅ Server restarted successfully! Status: {response.status_code}")
            break
        except:
            print(f"🔄 Restart attempt {attempt + 1}/3...")
            time.sleep(5)
    else:
        print("❌ Server restart failed")

except Exception as e:
    print(f"❌ Error testing server: {e}")

# Test ngrok tunnel if it exists
if 'public_url' in globals():
    print(f"\n🔍 Testing ngrok tunnel: {public_url}")
    try:
        response = requests.get(f"{public_url}/health", timeout=30)
        print(f"✅ Ngrok tunnel is working! Status: {response.status_code}")
    except Exception as e:
        print(f"❌ Ngrok tunnel test failed: {e}")
        print("💡 Try running the server setup cell again")
else:
    print("\n⚠️ No ngrok tunnel found. Run the server setup cell first.")

print("\n" + "="*50)
print("💡 TROUBLESHOOTING TIPS:")
print("="*50)
print("1. If local server fails: Restart runtime and run all cells again")
print("2. If ngrok fails: Check your auth token is set correctly")  
print("3. If still failing: Try running cells one by one with delays")
print("4. For memory issues: Enable High-RAM runtime")
print("5. For async errors: Make sure nest_asyncio is installed")
print("="*50)

## Step 7: Test the API (Optional)

In [ ]:
# Test the API directly from the notebook
import requests
import json
from IPython.display import Image as IPImage
from io import BytesIO

def test_generation(prompt, num_steps=25):
    """Test image generation directly."""
    print(f"Testing generation with prompt: {prompt}")
    
    # Check if we have the public URL
    if 'public_url' not in globals():
        print("❌ Error: No public URL found. Run the server setup cell first.")
        return None
    
    # Ensure we have a proper URL string
    test_url = public_url if isinstance(public_url, str) else str(public_url)
    if not test_url.startswith(('http://', 'https://')):
        print("❌ Error: Invalid URL format")
        return None

    try:
        print(f"🌐 Making request to: {test_url}")
        
        # Make API request
        response = requests.post(
            f"{test_url}/generate",
            json={
                "prompt": prompt,
                "num_steps": num_steps,
                "guidance_scale": 7.5
            },
            timeout=300  # 5 minute timeout for generation
        )

        if response.status_code == 200:
            result = response.json()
            print(f"✅ Generation completed in {result['generation_time']:.1f} seconds")

            # Decode and display image
            img_data = base64.b64decode(result['image_base64'])
            return IPImage(img_data)
        else:
            print(f"❌ Error: {response.status_code} - {response.text}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Wait a moment for everything to be ready
print("⏳ Waiting for services to be ready...")
time.sleep(5)

# Test with a simple prompt
print("🎨 Testing image generation...")
test_image = test_generation("a cute cat sitting in a garden", num_steps=20)
if test_image:
    display(test_image)
    print("🎉 Success! Your text-to-image service is working!")
else:
    print("🔄 If the test failed, try running the troubleshooting cell above.")
    print("💡 Common solutions:")
    print("   - Wait a bit longer (model might still be loading)")
    print("   - Run the troubleshooting cell to check server status")
    print("   - Restart runtime and run all cells again")

## Step 8: Keep the Service Running

In [ ]:
# DEBUGGING: Test image generation with detailed output
import requests
import base64
import subprocess
from PIL import Image
import io

def debug_generation():
    """Debug test with corrected URL extraction"""
    print("🔍 DEBUG: Testing image generation with detailed output...")
    
    # Get the correct ngrok URL (just the HTTPS part)
    try:
        result = subprocess.run(['ngrok', 'tunnel', 'list'], capture_output=True, text=True)
        
        # Extract just the HTTPS URL from the tunnel list
        ngrok_url = None
        for line in result.stdout.split('\n'):
            if 'https://' in line and 'localhost:8000' in line:
                # Extract just the https:// URL part
                parts = line.split()
                for part in parts:
                    if part.startswith('https://') and '.ngrok-free.dev' in part:
                        ngrok_url = part.strip()
                        break
                break
        
        if not ngrok_url:
            print("❌ No active ngrok tunnel found!")
            print("Run: !ngrok http 8000 --log stdout")
            return
        
        print(f"✅ Using ngrok URL: {ngrok_url}")
        
        # Test the /generate endpoint
        url = f"{ngrok_url}/generate"
        
        test_data = {
            "prompt": "a simple red circle on white background, minimal, clean",
            "num_steps": 10,  # Minimal steps for faster testing
            "guidance_scale": 7.5
        }
        
        print(f"🔍 Testing URL: {url}")
        print(f"📝 Request data: {test_data}")
        
        response = requests.post(
            url,
            json=test_data,
            headers={'Content-Type': 'application/json'},
            timeout=120
        )
        
        print(f"📊 Response status: {response.status_code}")
        print(f"📋 Response headers: {dict(response.headers)}")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Generation successful!")
            print(f"🎯 Prompt: {result.get('prompt', 'N/A')}")
            print(f"⏱️ Time: {result.get('generation_time', 'N/A')} seconds")
            
            # Test image decoding
            if 'image_base64' in result:
                try:
                    img_data = base64.b64decode(result['image_base64'])
                    img = Image.open(io.BytesIO(img_data))
                    print(f"🖼️ Image size: {img.size}")
                    print(f"🎨 Image mode: {img.mode}")
                    print("✅ Image successfully decoded!")
                    
                    # Display the image
                    from IPython.display import Image as IPImage
                    display(IPImage(img_data))
                    return True
                except Exception as e:
                    print(f"❌ Image decode error: {e}")
                    return False
            else:
                print("❌ No image_base64 in response")
                return False
        else:
            print(f"❌ Request failed: {response.status_code}")
            try:
                error_detail = response.json()
                print(f"📄 Error details: {error_detail}")
            except:
                print(f"📄 Response text: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Debug test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run the debug test
print("🚀 Running corrected debug test...")
debug_success = debug_generation()

if debug_success:
    print("\n🎉 Debug test completed successfully!")
    print("✅ Your ngrok tunnel and image generation are working properly!")
else:
    print("\n💡 Debug test failed. Common solutions:")
    print("1. Make sure ngrok tunnel is active: !ngrok tunnel list")
    print("2. If no tunnel, create one: !ngrok http 8000 --log stdout")
    print("3. Check if the FastAPI server is running on port 8000")
    print("4. If server stopped, re-run the server setup cell")
    print("5. Replace the old ngrok URL in your web interface with the new one")